In [140]:
import pandas as pd
import numpy as np
from fredapi import Fred
from config import key
from tqdm import tqdm, tqdm_notebook
import warnings
import time
warnings.filterwarnings('ignore')

In [133]:
#define FRED API call
fred = Fred(api_key= key)

In [134]:
#set column view options
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', -1)

In [ ]:
## Code to search FRED database
# search = fred.search('new home sales', order_by='popularity', sort_order='desc')
# search_df = pd.DataFrame(search[['title', 'id', 'popularity', 'units','seasonal_adjustment_short']])
# search_df[:20]

In [69]:
#desired indicators
indicators = ['USSLIND', 'UMCSENT', 'CSCICP03USM665S', 'INDPRO', 'TCU','DGORDER', 'NEWORDER', 'PI','BUSINV', 'TOTALSA',
             'MARTSMPCSM44X72USS', 'PAYEMS', 'UNRATE', 'ICSA', 'CCSA', 'CC4WSA', 'NPPTTL', 'CES0500000003', 'HSN1F',
             'EXHOSLUSM495S', 'PERMIT', 'CSUSHPINSA']

In [173]:
len(indicators)

24

In [172]:
#function to create dataframe and csv's for each indicator
errors=[]
for indicator in tqdm_notebook(indicators):
    try:
         #get data and info series
        data = fred.get_series(indicator, observation_start='1/31/1990')
        info = fred.get_series_info(indicator)
        #store as df's and reset indexes 
        data_df = pd.DataFrame(data)
        data_df.reset_index(level=0, inplace=True)
        info_df = pd.DataFrame(info)
        info_df.reset_index(level=0, inplace=True)
        #pivot info_df
        info_df.reset_index(level=0, inplace=True)
        df_dict = dict(zip(info_df['index'],info_df[0]))
        info_df = pd.DataFrame(df_dict,  index=[0])
        #duplicate info_df rows to equal data_df length
        info_df = pd.concat([info_df]*(len(data_df)), ignore_index=True)
        #join df's
        combine_df = data_df.join(info_df)
        #rename columns
        combine_df.rename(columns = {'index':'date', 0:'data'}, inplace = True) 
        #create new 'change from last column'
        combine_df.insert(2, 'change', np.nan)
        #create data for 'change' column
        changes = [combine_df.iloc[i,2]-combine_df.iloc[i-1,2] for i in range (1,len(combine_df))]
        change_clean = [float("{0:.3f}".format(x)) for x in changes]
        combine_df['change'][1:len(combine_df)] = change_clean
        #write to csv
        combine_df.to_csv(f"data/{indicator}.csv",index = False, header = True)  
        time.sleep(0.2)
        print(f'{indicator} retrieved')
    except: 
        print(f'error with {indicator} data. Proceeding...')
        errors.append(indicator)
        
    

USSLIND retrieved
UMCSENT retrieved
CSCICP03USM665S retrieved
INDPRO retrieved
TCU retrieved
DGORDER retrieved
NEWORDER retrieved
PI retrieved
BUSINV retrieved
TOTALSA retrieved
MARTSMPCSM44X72USS retrieved
PAYEMS retrieved
UNRATE retrieved
ICSA retrieved
CCSA retrieved
CC4WSA retrieved
NPPTTL retrieved
CES0500000003 retrieved
HSN1F retrieved
EXHOSLUSM495S retrieved
PERMIT retrieved
error with M0264AUSM500NNBR data. Proceeding...
error with M0263AUSM500NNBR data. Proceeding...
CSUSHPINSA retrieved

The following data contained errors and did not save ['M0264AUSM500NNBR', 'M0263AUSM500NNBR']


In [175]:
combine_df

,date,data,change,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,1990-01-01,76.527,NaN,CSUSHPINSA,2020-02-18,2020-02-18,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2019-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2020-01-28 08:22:02-06,86,"For more information regarding the index, please visit Standard & Poor's (http://us.spindices.com/index-family/real-estate/sp-corelogic-case-shiller). There is more information about home price sales pairs in the Methodology section. Copyright, 2016, Standard & Poor's Financial Services LLC. Reprinted with permission."
1,1990-02-01,76.587,NaN,CSUSHPINSA,2020-02-18,2020-02-18,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2019-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2020-01-28 08:22:02-06,86,"For more information regarding the index, please visit Standard & Poor's (http://us.spindices.com/index-family/real-estate/sp-corelogic-case-shiller). There is more information about home price sales pairs in the Methodology section. Copyright, 2016, Standard & Poor's Financial Services LLC. Reprinted with permission."
2,1990-03-01,76.790,NaN,CSUSHPINSA,2020-02-18,2020-02-18,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2019-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2020-01-28 08:22:02-06,86,"For more information regarding the index, please visit Standard & Poor's (http://us.spindices.com/index-family/real-estate/sp-corelogic-case-shiller). There is more information about home price sales pairs in the Methodology section. Copyright, 2016, Standard & Poor's Financial Services LLC. Reprinted with permission."
3,1990-04-01,77.041,NaN,CSUSHPINSA,2020-02-18,2020-02-18,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2019-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2020-01-28 08:22:02-06,86,"For more information regarding the index, please visit Standard & Poor's (http://us.spindices.com/index-family/real-estate/sp-corelogic-case-shiller). There is more information about home price sales pairs in the Methodology section. Copyright, 2016, Standard & Poor's Financial Services LLC. Reprinted with permission."
4,1990-05-01,77.299,NaN,CSUSHPINSA,2020-02-18,2020-02-18,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2019-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2020-01-28 08:22:02-06,86,"For more information regarding the index, please visit Standard & Poor's (http://us.spindices.com/index-family/real-estate/sp-corelogic-case-shiller). There is more information about home price sales pairs in the Methodology section. Copyright, 2016, Standard & Poor's Financial Services LLC. Reprinted with permission."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,2019-07-01,211.611,NaN,CSUSHPINSA,2020-02-18,2020-02-18,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2019-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2020-01-28 08:22:02-06,86,"For more information regarding the index, please visit Standard & Poor's (http://us.spindices.com/index-family/real-estate/sp-corelogic-case-shiller). There is more information about home price sales pairs in the Methodology section. Copyright, 2016, Standard & Poor's Financial Services LLC. Reprinted with permission."
355,2019-08-01,211.987,NaN,CSUSHPINSA,2020-02-18,2020-02-18,S&P/Case-Shiller U.S. National Home Price Index,1987-01-01,2019-11-01,Monthly,M,Index Jan 2000=100,Index Jan 2000=100,Not Seasonally Adjusted,NSA,2020-01-28 08:22:02-06,86,"For more information regarding the index, please visit Standard & Poor's (http://us.spindices.com/index-family/real-estate/sp-corelogic-case-shiller). There is more informat

## Examples
s = fred.get_series('SP500', observation_start='1/31/2014')
info = fred.get_series_info('PAYEMS')
info['title']
df = fred.search_by_category(101, limit=10, order_by='popularity', sort_order='desc')
df['title']